# Nutrition Database

## Goal

- [ ] Research with Copilot / ChatGPT how to attain this...
    - [ ] Do I want to scrape?
    - [ ] Do I want to API?
- [ ] Test it out on the below output from recipe API...

In [1]:
# Recipe output from Ninja API
[{'title': 'Energy Bars, Banana/maple/walnut [Vegan]', 'ingredients': ['2 Bananas; over-ripe', '3 tb Maple syrup', '3 tb Walnuts; toasted', '2 tb Flour; vital gluten', '1 1/3 c Rolled oats', '1/2 c Oat bran', '1 ts Vanilla extract', '2 ts Lecithin; granules', '1/4 ts Salt'], 'servings': '6 Servings', 'instructions': 'Combine all ingredients in cusinart and blend until a nearly smooth paste. Spread in 8"x8" or 9"x9" pan or drop on cookie sheet and shape into "cliff bar" style bars. Bake at 350 for 15 minutes. Let cool completely before eating. NOTE: these are not as "shiny" as the other style energy bars. 210 calories per 1/6th recipe/per bar fat: 5.9g sat. fat: .8g cholesterol: 0 carbohydrate: 34.9g dietary fiber:4.4g protein: 9.5g sodium: 9.mg'}]

[{'title': 'Energy Bars, Banana/maple/walnut [Vegan]',
  'ingredients': ['2 Bananas; over-ripe',
   '3 tb Maple syrup',
   '3 tb Walnuts; toasted',
   '2 tb Flour; vital gluten',
   '1 1/3 c Rolled oats',
   '1/2 c Oat bran',
   '1 ts Vanilla extract',
   '2 ts Lecithin; granules',
   '1/4 ts Salt'],
  'servings': '6 Servings',
  'instructions': 'Combine all ingredients in cusinart and blend until a nearly smooth paste. Spread in 8"x8" or 9"x9" pan or drop on cookie sheet and shape into "cliff bar" style bars. Bake at 350 for 15 minutes. Let cool completely before eating. NOTE: these are not as "shiny" as the other style energy bars. 210 calories per 1/6th recipe/per bar fat: 5.9g sat. fat: .8g cholesterol: 0 carbohydrate: 34.9g dietary fiber:4.4g protein: 9.5g sodium: 9.mg'}]

In [3]:
# TODO Since it's a JSON file, do I need to transform it somehow in order to process it?

# Nutrition Database — Actionable To‑Do List

What I did
- Converted the previous guidance into a focused, action‑oriented to‑do list that only covers the nutritional database approach. This file is structured so you can pick tasks top-to-bottom and make measurable progress.

How to use this list
- Work in order unless you have a specific need (e.g., you already have a preferred DB).
- Each item includes a short why, concrete steps, estimated effort, and acceptance criteria.
- Mark checkboxes as you complete items.

Priority legend
- P0 = immediate / unblocker
- P1 = essential next steps
- P2 = useful improvements
- P3 = optional / future

--------------------------------------------------------------------------------
P0 — Decide core approach & tooling (critical first decision)
- [ ] Choose data sources (pick at least one baseline + one product source)
  - Options: USDA FoodData Central (baseline, public domain), Open Food Facts (barcoded products, ODbL), Commercial API (Nutritionix/Edamam/Spoonacular) for branded coverage.
  - Effort: 10–20m
  - Acceptance: decision recorded in README or this file.

- [ ] Choose a local storage backend
  - Recommended for notebooks: DuckDB (zero‑admin, fast for bulk analytics). Alternative: SQLite (simple) or PostgreSQL (production).
  - Effort: 10–20m
  - Acceptance: backend choice noted and one empty DB file created (e.g., data/nutrition.duckdb).

--------------------------------------------------------------------------------
P0 — Acquire official bulk datasets (legal & reproducible)
- [ ] Download USDA FoodData Central bulk files (JSON/CSV)
  - Steps:
    - Fetch latest release from https://fdc.nal.usda.gov/ or FDC bulk download page.
    - Save source file(s) under data/raw/usda/ with a snapshot date and filename.
  - Effort: 15–30m
  - Acceptance: file(s) present in data/raw/usda/ and a metadata note (URL + date).

- [ ] Download Open Food Facts product dump (optional but highly recommended)
  - Steps:
    - Fetch product dump (country/global) from Open Food Facts dumps.
    - Save under data/raw/off/ and record license (ODbL).
  - Effort: 15–30m
  - Acceptance: file(s) present in data/raw/off/ with metadata & license note.

- [ ] (Optional) Evaluate a commercial API for branded coverage
  - Steps:
    - Create free/trial accounts with Nutritionix, Edamam, or Spoonacular; check request limits and pricing.
    - Document API capabilities and cost.
  - Effort: 30–60m
  - Acceptance: a short comparison note in data_catalog/README.

--------------------------------------------------------------------------------
P1 — Ingestion & canonical schema
- [ ] Define a minimal canonical schema for food items
  - Suggested fields:
    - source (usda | off | api:name)
    - source_id (fdc_id | barcode | api_id)
    - name (primary description)
    - normalized_name (lowercased, stripped)
    - category (e.g., "Fruits", "Dairy")
    - serving_sizes (list of {label, grams})
    - nutrients (structured JSON or normalized nutrient columns; include kcal, protein, fat, carbs, fiber, sugar, sodium at minimum)
    - last_updated (snapshot date)
  - Effort: 30–60m
  - Acceptance: schema document saved at data_catalog/schema.md

- [ ] Implement ingestion script for USDA → local DB
  - Steps:
    - Read USDA JSON/CSV.
    - Extract fields into canonical schema.
    - Normalize nutrient basis (convert per-serving to per-100g if necessary).
    - Write into DuckDB table `foods_usda` (or chosen backend).
  - Effort: 2–4 hours
  - Acceptance: DB contains `foods_usda` with sample records and correct nutrient columns.

- [ ] Implement ingestion script for Open Food Facts → local DB
  - Steps:
    - Read OFF JSON/CSV.
    - Map nutriments and barcodes to canonical schema.
    - Write into DuckDB table `foods_off`.
    - Record OFF license/attribution metadata in DB or data_catalog.
  - Effort: 2–4 hours
  - Acceptance: DB contains `foods_off` with sample records and license noted.

- [ ] Add ingestion metadata tracking
  - Steps:
    - Store dataset filename, source URL, download date, file SHA/size in a `data_snapshots` table or YAML metadata.
  - Effort: 30–60m
  - Acceptance: `data_snapshots` populated after each import.

--------------------------------------------------------------------------------
P1 — Build core lookup & matching pipeline
- [ ] Create normalized-name utilities (normalization rules)
  - Steps:
    - Lowercase, strip punctuation, remove parenthetical text, collapse multi-spaces.
    - Maintain synonyms map (e.g., "scallions" → "green onion").
  - Effort: 1–2 hours
  - Acceptance: normalized_name("1 cup chopped Onion (peeled)") → "onion"

- [ ] Implement full-text search index for names
  - Steps:
    - Use DuckDB FTS or SQLite FTS (or create simple token index with trigram).
    - Populate index with normalized names and common synonyms.
  - Effort: 1–3 hours
  - Acceptance: fast keyword search returns candidates.

- [ ] Implement fuzzy-matching fallback using rapidfuzz
  - Steps:
    - Query FTS for candidates, score with rapidfuzz, return top N.
    - Define confidence thresholds (e.g., >90% exact, 70–90% candidate, <70% low).
  - Effort: 1–2 hours
  - Acceptance: function find_candidates("brown sugar") returns sensible top-3 matches with scores.

- [ ] Add match-selection & provenance
  - Steps:
    - For each match, return: source, source_id, name, normalized_name, score.
    - If ambiguous, surface top N or require human confirmation in notebook/CLI.
  - Effort: 1–2 hours
  - Acceptance: outputs include provenance and confidence.

--------------------------------------------------------------------------------
P1 — Unit conversion, density table & ingredient parsing (minimal)
- [ ] Create a small density/serving lookup table
  - Steps:
    - CSV or JSON mapping like {item: {cup_g: X, tbsp_g: Y, piece_g: Z, density_g_per_ml: Z}} for top ~40 common items (flour, sugar, oil, rice, butter, milk, vegetables, fruits).
    - Source values from USDA, published cookbooks, or measured standards; record sources.
  - Effort: 2–4 hours
  - Acceptance: density table present at data/densities.csv and a README entry describing sources.

- [ ] Implement unit conversion util: to_grams(qty, unit, normalized_name)
  - Steps:
    - Use pint or a small custom mapping to convert units to grams using densities and per-servings.
    - Fallback: if no density available, warn and use category-average or request user selection.
  - Effort: 2–4 hours
  - Acceptance: to_grams("1", "cup", "all-purpose flour") ≈ 120g (testable sample).

- [ ] Minimal ingredient parsing helper
  - Steps:
    - Lightweight parser (regex + heuristics) that extracts quantity, unit, and item text from a recipe line.
    - Produce output suitable to feed to to_grams() and matching pipeline.
  - Effort: 2–4 hours
  - Acceptance: parse_ingredient("1 1/2 cups chopped carrots") → {qty:1.5, unit:"cup", name:"carrots"}

--------------------------------------------------------------------------------
P1 — Nutrient scaling, summation & confidence
- [ ] Implement nutrient scaling function
  - Steps:
    - Standard formula: nutrient_total = (grams / reference_grams) * nutrient_value (make sure nutrient units match, e.g., mg/g).
    - Normalize nutrient basis in DB entries (prefer per 100 g).
  - Effort: 1–2 hours
  - Acceptance: Unit tests show correct scaling for sample food entry.

- [ ] Sum recipe-level nutrients and attach provenance & confidence per ingredient
  - Steps:
    - For each ingredient: grams, matched source, score, per-ingredient nutrients.
    - Sum nutrients across ingredients; produce a result payload with breakdown and flags for low-confidence matches.
  - Effort: 1–2 hours
  - Acceptance: example run for a small recipe returns totals + per-ingredient provenance.

--------------------------------------------------------------------------------
P2 — Caching & hybrid strategy for branded items
- [ ] Implement local caching layer for API lookups
  - Steps:
    - If using a commercial API for branded products, cache responses in local DB to reduce cost and latency.
  - Effort: 1–2 hours
  - Acceptance: cache hit reduces API calls; cache table contains API responses + timestamp.

- [ ] Decision rule for hybrid lookup
  - Steps:
    - Pipeline: attempt exact DB match → FTS/fuzzy → if low confidence and API available, fall back to API → cache result.
  - Effort: 1 hour
  - Acceptance: documented in pipeline README.

--------------------------------------------------------------------------------
P2 — Provenance, licensing & governance
- [ ] Create data_catalog/README with:
  - List of sources and exact download URLs
  - Snapshot dates and file checksums
  - License obligations (USDA public domain, OFF ODbL share‑alike, commercial API terms)
  - Effort: 1–2 hours
  - Acceptance: data_catalog/README added to repo.

- [ ] Ensure each DB record retains source and source_id field
  - Effort: 30–60m
  - Acceptance: every row has source and source_id filled.

--------------------------------------------------------------------------------
P2 — Tests & reproducibility
- [ ] Add unit tests for:
  - Ingestion (sample fixture)
  - Normalization / matching
  - to_grams conversion
  - Nutrient scaling
  - Effort: 3–6 hours
  - Acceptance: tests run locally and cover core cases.

- [ ] Create a small example notebook showing ingestion → match → calculation
  - Effort: 2–4 hours
  - Acceptance: notebook executes end-to-end on a fresh environment using the local DB.

--------------------------------------------------------------------------------
P3 — Enhancements (future)
- [ ] Add retention factors / cooked vs raw conversions (USDA retention factors)
- [ ] Expand density table coverage (crowdsource corrections)
- [ ] Add embedding/semantic matching for hard cases
- [ ] Add CLI or tiny API for external usage
- [ ] Periodic automated dataset refresh (cron / GitHub Actions) with snapshot audits

--------------------------------------------------------------------------------
Suggested immediate next actions (pick one)
- A: Create data/raw/ and download USDA + OFF snapshots and record metadata (P0+P1 ingest prep)
- B: Scaffold ingestion script for USDA into DuckDB with canonical schema (P1 ingestion)
- C: Create density CSV and implement to_grams + a few unit tests (P1 conversions)

Tell me which immediate action you want and I will generate the first script or file for that task.